# Rarity Analyse 
**created by yuteng_zeng on 4/23/2023**

In [1]:
# install required library
#!pip install web3

In [2]:
# import libraries
import requests
import json
from web3 import Web3

import numpy as np
import os
import time
import h5py

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

import matplotlib.pyplot as plt

import pickle

In [3]:
# get api_interfaces by tangyizhi
# API factors
API_key = 'PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'
API_url = "https://eth-mainnet.g.alchemy.com/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP"
API_nft_url = 'https://eth-mainnet.g.alchemy.com/nft/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'

# data collection functions
def getNFTsForCollection(contractAddress, withMetadata):
    url = API_url + '/getNFTsForCollection?contractAddress=' + contractAddress + '&withMetadata=' + str(withMetadata)
    r = requests.get(url, headers={"accept": "application/json"})
    return r
def getNFTSales(contractAddress, tokenid):
    # from block 0 to latest block, with ascending order
    url = API_url + '/getNFTSales?fromBlock=0&toBlock=latest&order=asc&contractAddress=' + contractAddress + '&tokenId=' + str(tokenid)
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getFloorPrice(contractAddress):
    url = API_nft_url + "/getFloorPrice?contractAddress=" + contractAddress
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def computeRarity(contractAddress, tokenid):
    url = API_nft_url + '/computeRarity/?contractAddress=' + contractAddress + '&tokenId=' + str(tokenid)
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getMetaData(contractAddress, tokenid):
    url = API_nft_url + '/getNFTMetadata?contractAddress=' + contractAddress + '&tokenId=' + str(tokenid) + '&refreshCache=false'
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getNFTAttributes(contractAddress):
    url = API_nft_url + '/summarizeNFTAttributes?contractAddress=' + contractAddress
    r = requests.get(url, headers={"accept": "application/json"})
    return r

*The following is api test and further utils for data collection and prep*

In [4]:
response = json.loads(getNFTsForCollection('0x60E4d786628Fea6478F785A6d7e704777c86a7c6', True).content)
print(response['nfts'][0])

{'contract': {'address': '0x60e4d786628fea6478f785a6d7e704777c86a7c6'}, 'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000000000', 'tokenMetadata': {'tokenType': 'ERC721'}}, 'title': '', 'description': '', 'tokenUri': {'gateway': 'https://boredapeyachtclub.com/api/mutants/0', 'raw': 'https://boredapeyachtclub.com/api/mutants/0'}, 'media': [{'gateway': 'https://nft-cdn.alchemy.com/eth-mainnet/c480d814f8a4e2b5f02dd3aa189ca742', 'thumbnail': 'https://res.cloudinary.com/alchemyapi/image/upload/thumbnailv2/eth-mainnet/c480d814f8a4e2b5f02dd3aa189ca742', 'raw': 'ipfs://QmURua8bNrAwX76Tp6G9t6Lospdxyt61JGy3UsXY7skfR1', 'format': 'png', 'bytes': 577530}], 'metadata': {'image': 'ipfs://QmURua8bNrAwX76Tp6G9t6Lospdxyt61JGy3UsXY7skfR1', 'attributes': [{'value': 'M1 Purple', 'trait_type': 'Background'}, {'value': 'M1 Cheetah', 'trait_type': 'Fur'}, {'value': 'M1 Scumbag', 'trait_type': 'Eyes'}, {'value': 'M1 Toga', 'trait_type': 'Clothes'}, {'value': 'M1 Bored Unshaven', 

In [5]:
# check the attribute lists.
response = json.loads(getNFTAttributes('0x60E4d786628Fea6478F785A6d7e704777c86a7c6').content)
print(response['summary'].keys())

dict_keys(['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name'])


In [6]:
response = json.loads(computeRarity('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',0).content)
print(response)

[{'value': 'M1 Purple', 'trait_type': 'Background', 'prevalence': 0.09621621621621622}, {'value': 'M1 Cheetah', 'trait_type': 'Fur', 'prevalence': 0.03099099099099099}, {'value': 'M1 Scumbag', 'trait_type': 'Eyes', 'prevalence': 0.016885456885456885}, {'value': 'M1 Toga', 'trait_type': 'Clothes', 'prevalence': 0.01662805662805663}, {'value': 'M1 Bored Unshaven', 'trait_type': 'Mouth', 'prevalence': 0.11613899613899614}]


In [7]:
response = json.loads(getNFTSales('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',555).content)
print(response['nftSales'][-1]['sellerFee'])
print(response['nftSales'][-1]['protocolFee'])
print(response['nftSales'][-1]['royaltyFee'])

{'amount': '23740500000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}
{'amount': '624750000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}
{'amount': '624750000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}


In [8]:
# get the sell price from the api
def get_price(contractAddress, tokenid, roundto):
    response = json.loads(getNFTSales(contractAddress,tokenid).content)
    res = int(response['nftSales'][-1]['sellerFee']['amount'])/ (10**int(response['nftSales'][-1]['sellerFee']['decimals']))
    res = round(res, roundto)
    return res

print(get_price('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',555,5))
print(get_price('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',28905,5))

23.7405
106.875


In [9]:
# get feature and labels for building the dataset
def get_monkey(contractAddress, tokenid, roundto):
    # initialize all attributes to one, which means least rare
    feature = {'Fur':1, 'Eyes':1, 'Background':1, 'Mouth':1, 'Clothes':1, 'Earring':1, 'Hat':1, 'Name':1}
    feature_response = json.loads(computeRarity(contractAddress,tokenid).content)
    #print(feature_response)
    for i in feature_response:
        feature[i['trait_type']] = round(1 / i['prevalence'], roundto)
        
    # get price
    price = None
    try:
        price = get_price(contractAddress, tokenid, roundto)
    except:
        pass
    return feature, price

there are monkeys that has never been traded, thus *try* get the price, 

if price donnot exist then the price is set to **None**

In [10]:
print(get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',2,5))
print(get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',10000,5)[1] is None)

({'Fur': 9.95643, 'Eyes': 7.71752, 'Background': 10.76177, 'Mouth': 5.86327, 'Clothes': 33.31904, 'Earring': 1, 'Hat': 99.61538, 'Name': 1}, 20.9)
True


make the dataset using the template ['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name'] as **features**

seller price as **labels** to predict


In [11]:
# now gathering the data and build dataset
'''
attribute_list = ['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name']
features = []
labels = []
token_id = 1
while len(labels) <= 150:    
    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)
    if price is not None:
        features.append([feature[x] for x in attribute_list])
        labels.append(price)
    token_id += 1
    time.sleep(0.01)
print(features, labels)
'''

"\nattribute_list = ['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name']\nfeatures = []\nlabels = []\ntoken_id = 1\nwhile len(labels) <= 150:    \n    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)\n    if price is not None:\n        features.append([feature[x] for x in attribute_list])\n        labels.append(price)\n    token_id += 1\n    time.sleep(0.01)\nprint(features, labels)\n"

In [12]:
# only runone time to get data
# partially acquire data... getting too fast would cause api denial
'''
while len(labels) < 1200:    
    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)
    if price is not None:
        features.append([feature[x] for x in attribute_list])
        labels.append(price)
    token_id += 1
    time.sleep(0.01)
print(features, labels)
'''

"\nwhile len(labels) < 1200:    \n    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)\n    if price is not None:\n        features.append([feature[x] for x in attribute_list])\n        labels.append(price)\n    token_id += 1\n    time.sleep(0.01)\nprint(features, labels)\n"

In [13]:
#print(token_id)

In [14]:
#print(len(features), len(labels))

In [15]:
# store data in a pickle file
'''
data = []
data.append(features)
data.append(labels)
len(data[1])
with open('data_features_labels.pkl','wb') as f:
    pickle.dump(data, f)
'''

"\ndata = []\ndata.append(features)\ndata.append(labels)\nlen(data[1])\nwith open('data_features_labels.pkl','wb') as f:\n    pickle.dump(data, f)\n"

In [16]:
with open('data_features_labels.pkl','rb') as f:
    load_data = pickle.load(f)
len(load_data[1])

1200

With the data acquired, we can directly apply ML models to fit the data and do predictions, we can do so by **RandomForest**, which may be suitable for our data since the feature dimension is not large and can be seperately judged, perhaps. For example:

In [17]:
# define feature and label
seed = 98
np.random.seed(seed)
X = load_data[0]
Y = load_data[1]
# split data into train and test sets
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X,Y,test_size = 0.1, random_state = seed)
# define randomforest model
RF = RandomForestRegressor(n_estimators = 100, random_state = seed)
# normalize data
scaler = StandardScaler()
scaler.fit(X_TRAIN)
# apply normalization to train and test sets
X_TRAIN = scaler.transform(X_TRAIN)
X_TEST = scaler.transform(X_TEST)
RF.fit(X_TRAIN, Y_TRAIN)
# make prediction on test set
pred = RF.predict(X_TEST)
# analyse model function
mse = mean_squared_error(Y_TEST, pred)
print('showing some of the results:')
print('preds:', pred[:20])
print('labels:', Y_TEST[:20])
print('MSE:', mse)

showing some of the results:
preds: [15.4504133 20.0809271 10.7115035 12.9803584 19.218093  15.3320771
 13.2271288 38.153786  13.6505272 13.3473243 12.5995639 17.5792
 13.4893934 12.4509482 19.2247025 14.7334061 17.4171825 11.3171323
 11.7479038 21.4716115]
labels: [15.2, 5.225, 4.4175, 16.55375, 19.95, 19.0, 13.2692, 39.2, 16.235, 20.9, 5.8425, 15.2, 15.66455, 24.4055, 17.1, 8.5405, 9.785, 14.9625, 7.125, 29.925]
MSE: 1208007.0499351316


It seems that the features have more high-dimension relations, which can hardly be interpreted as decisiontree classification. We can do a **SVM** instead, to get more high-dimension features, as following:

In [18]:
svr = SVR(kernel = 'rbf', C=1, gamma ='scale')
svr.fit(X_TRAIN, Y_TRAIN)
# make prediction on test set
pred = svr.predict(X_TEST)
# analyse model function
mse = mean_squared_error(Y_TEST, pred)
print('showing some of the results:')
print('preds:', pred[:20])
print('labels:', Y_TEST[:20])
print('MSE:', mse)

showing some of the results:
preds: [13.97448599 16.50242358 13.79753067 14.0856291  16.28839131 17.11522885
 14.48781127 18.647618   15.30622184 17.76094566 14.53069409 18.62429624
 14.39255615 14.22981915 14.97823383 14.08298295 13.77782817 15.08508179
 14.1815663  19.13894446]
labels: [15.2, 5.225, 4.4175, 16.55375, 19.95, 19.0, 13.2692, 39.2, 16.235, 20.9, 5.8425, 15.2, 15.66455, 24.4055, 17.1, 8.5405, 9.785, 14.9625, 7.125, 29.925]
MSE: 115.47965802546605


Much better!

Or, we can build neural networks for this task:

Perhaps starting with a **MLP** model.

Then we need to implement the basic nn pipeline, including dataset management, network definition, training and testing. 

In [19]:
# load data for train/validate/test datasets
feature_train = []
label_train = []
feature_validation = []
label_validation = []
feature_test = []
label_test = []

data_length = len(load_data[1])
print(data_length)
# train/validation/test = 8:1:1
# divide id into 10 groups, to eliminate influence by time 
group = 10
group_l = data_length/group
for i in range(group):
    for j in range(int(group_l)):
        if j < 0.8*group_l:
            feature_train.append(load_data[0][i * group + j])
            label_train.append(load_data[1][i * group + j])
        elif j < 0.9*group_l:
            feature_validation.append(load_data[0][i * group + j])
            label_validation.append(load_data[1][i * group + j])
        else:
            feature_test.append(load_data[0][i * group + j])
            label_test.append(load_data[1][i * group + j])
print(len(feature_train), len(label_train))
print(len(feature_validation), len(label_validation))
print(len(feature_test), len(label_test))

1200
960 960
120 120
120 120


In [20]:
# define datasets

tr_name = 'tr_set.hdf5'
val_name = 'val_set.hdf5'
test_name = 'test_set.hdf5'
'''
tr_dataset = h5py.File(tr_name, 'a')
val_dataset = h5py.File(val_name, 'a')
test_dataset = h5py.File(test_name, 'a')

# load data into the dataset
tr_dataset['feature'] = feature_train
tr_dataset['label'] = label_train
val_dataset['feature'] = feature_validation
val_dataset['label'] = label_validation
test_dataset['feature'] = feature_test
test_dataset['label'] = label_test

tr_dataset.close()
val_dataset.close()
test_dataset.close()
'''

"\ntr_dataset = h5py.File(tr_name, 'a')\nval_dataset = h5py.File(val_name, 'a')\ntest_dataset = h5py.File(test_name, 'a')\n\n# load data into the dataset\ntr_dataset['feature'] = feature_train\ntr_dataset['label'] = label_train\nval_dataset['feature'] = feature_validation\nval_dataset['label'] = label_validation\ntest_dataset['feature'] = feature_test\ntest_dataset['label'] = label_test\n\ntr_dataset.close()\nval_dataset.close()\ntest_dataset.close()\n"

In [21]:
# define dataloader
batch_size = 30

class dataset_pipeline(Dataset):
    def __init__(self, path):
        super(dataset_pipeline, self).__init__()
        self.h5pyLoader = h5py.File(path, 'r')
        self.feature = self.h5pyLoader['feature']
        self.label = self.h5pyLoader['label']
        self.len_ = len(self.label)
        
    def __getitem__(self, index):
        feature_item = torch.from_numpy(np.array(self.feature[index]).astype(np.float32))
        label_item = self.label[index]
        return feature_item, label_item
    
    def __len__(self):
        return self.len_

In [22]:
# Load data from the datasets
train_loader = DataLoader(dataset_pipeline(tr_name),
                          batch_size = batch_size,
                          shuffle = True
                         )
val_loader = DataLoader(dataset_pipeline(val_name),
                          batch_size = batch_size,
                          shuffle = False
                         )
test_loader = DataLoader(dataset_pipeline(test_name),
                          batch_size = batch_size,
                          shuffle = False
                         )

In [23]:
# define the MLP model
class RP_MLP(nn.Module):
    def __init__(self, feature_dim):
        super(RP_MLP, self).__init__()
        
        self.feature_dim = feature_dim
        self.hidden_unit = 16
        self.output = 1
        
        self.layer1 = nn.Sequential(nn.Linear(self.feature_dim, self.hidden_unit),
                                    nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(nn.Linear(self.hidden_unit, self.hidden_unit),
                                    nn.Sigmoid()
        )
        self.layer3 = nn.Linear(self.hidden_unit, self.output)

    def forward(self, input):
        
        output = self.layer1(input)
        output = self.layer2(output)
        output = self.layer3(output)
        return output

In [24]:
# train the model
def train(model, loader, opt, loss_fn):
    '''
    Training Step, called once per epoch.
    Model is updated.
    args:
        model: nn.Module
        loader: torch DataLoader
        opt: torch.optim
        loss_fn: differentiable loss_fn(probs, labels) -> scalar
    return:
        average training loss for this epoch
    '''
    model = model.train()
    train_loss_batch = []
    for batch_id, [data, labels] in enumerate(loader):
        opt.zero_grad()
        output = model(data)
        output = output.float()
        labels = labels.float()
        loss = loss_fn(output.flatten(), labels)       
        train_loss_batch.append(loss)
        loss.backward()
        opt.step()
    train_loss_epoch = sum(train_loss_batch) / len(train_loss_batch)
    return float(train_loss_epoch)

def validate(model, loader, criterion, return_results = False):
    '''
    Validation Step, called once per epoch.
    args:
        model: nn.Module
        loader: torch DataLoader
        criterion: criterion(predicts, labels) -> scalar
    return:
        average accuracy of all frames, and if return_results,
        (predicted probabalities, predicted labels)
    
    '''
    model = model.eval()  
    all_labels = torch.tensor([])
    all_preds = torch.tensor([])
    with torch.no_grad():
        for batch_id, [data, labels] in enumerate(loader):
            pred = model(data)
            all_labels = torch.cat([all_labels, labels])
            all_preds = torch.cat([all_preds, pred])
            
    val_acc_epoch = criterion(all_preds.flatten(), all_labels.flatten())
    if return_results:
        return float(val_acc_epoch), (all_labels, all_preds)
    else:
        return float(val_acc_epoch)

In [25]:
# initiate the model
rp_MLP = RP_MLP(feature_dim = 8)
# define optimizer
opt = optim.Adam(rp_MLP.parameters(), lr = 1e-3)
#hyperparas
total_epoch = 200
model_save = 'best_RP_MLP.pt'

# Main Func
train_loss = []
val_acc = []

for epoch in range(1, total_epoch + 1):
    train_loss_epoch = train(rp_MLP,train_loader, opt, F.mse_loss)
    val_acc_epoch,  detail = validate(rp_MLP,val_loader, F.mse_loss, True)
    #print(detail)
    train_loss.append(train_loss_epoch)
    val_acc.append(val_acc_epoch)
    print(f'Epoch No.{epoch}:')
    print(f'      Training loss: {str(round(train_loss_epoch, 2))}')
    print(f'      Validation loss: {str(round(val_acc_epoch, 2))}')
    
    if train_loss[-1] == np.min(train_loss):
        print('      Best training model found.')
    if val_acc[-1] == np.min(val_acc):
        # save current best model on validation set
        with open(model_save, 'wb') as f:
            torch.save(rp_MLP.state_dict(), f)
            print('      Best validation model found and saved.')
    
    print('-' * 99)

Epoch No.1:
      Training loss: 261.84
      Validation loss: 277.74
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.2:
      Training loss: 249.52
      Validation loss: 264.28
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.3:
      Training loss: 236.28
      Validation loss: 249.26
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.4:
      Training loss: 221.65
      Validation loss: 233.2
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.

Epoch No.35:
      Training loss: 59.81
      Validation loss: 57.07
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.36:
      Training loss: 59.2
      Validation loss: 56.3
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.37:
      Training loss: 58.72
      Validation loss: 55.62
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.38:
      Training loss: 58.34
      Validation loss: 54.9
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.39:
  

Epoch No.70:
      Training loss: 56.27
      Validation loss: 50.55
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.71:
      Training loss: 56.27
      Validation loss: 50.54
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.72:
      Training loss: 56.26
      Validation loss: 50.54
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.73:
      Training loss: 56.23
      Validation loss: 50.55
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.74:
      Training loss: 56.17
      Validation loss: 50.56
      Best training model found.


Epoch No.105:
      Training loss: 48.43
      Validation loss: 46.53
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.106:
      Training loss: 48.22
      Validation loss: 46.33
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.107:
      Training loss: 48.12
      Validation loss: 46.31
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.108:
      Training loss: 48.04
      Validation loss: 46.18
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No

Epoch No.141:
      Training loss: 44.15
      Validation loss: 41.24
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.142:
      Training loss: 44.07
      Validation loss: 41.28
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.143:
      Training loss: 43.9
      Validation loss: 41.03
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.144:
      Training loss: 43.83
      Validation loss: 40.8
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.145:
      Training loss: 43.75
      Validati

Epoch No.183:
      Training loss: 40.75
      Validation loss: 38.38
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.184:
      Training loss: 40.64
      Validation loss: 37.88
      Best training model found.
      Best validation model found and saved.
---------------------------------------------------------------------------------------------------
Epoch No.185:
      Training loss: 40.62
      Validation loss: 37.99
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.186:
      Training loss: 40.61
      Validation loss: 38.16
      Best training model found.
---------------------------------------------------------------------------------------------------
Epoch No.187:
      Training loss: 40.52
      Validation loss: 38.07
      Best training model found.
----------------------------------------

In [26]:
# model evaluation
def evaluation(model, loader, criterion, istest = False):
    all_labels = torch.tensor([])
    all_preds = torch.tensor([])
    with torch.no_grad():
        for batch_id, [data, labels] in enumerate(loader):
            pred = model(data)
            all_labels = torch.cat([all_labels, labels])
            all_preds = torch.cat([all_preds, pred])
            
    val_acc_epoch = criterion(all_preds.flatten(), all_labels.flatten())
    if istest:
        return float(val_acc_epoch), (all_labels, all_preds)
    else:
        return float(val_acc_epoch)
# Load datasets
train_loader = DataLoader(dataset_pipeline(tr_name),
                          batch_size = batch_size,
                          shuffle = False
                         )
val_loader = DataLoader(dataset_pipeline(val_name),
                          batch_size = batch_size,
                          shuffle = False
                         )
test_loader = DataLoader(dataset_pipeline(test_name),
                          batch_size = batch_size,
                          shuffle = False
                         )
# Load model weights
rp_MLP.load_state_dict(torch.load(model_save))

final_train_acc = evaluation(rp_MLP, train_loader, F.mse_loss)
final_val_acc = evaluation(rp_MLP, val_loader, F.mse_loss)
final_test_acc, details = evaluation(rp_MLP, test_loader, F.mse_loss, istest = True)
print('Prices are :\n', details[0], '\nWith their predictions as :\n', detail[1].flatten())
print(f'Final train loss {str(round(final_train_acc, 2))} | val loss {str(round(final_val_acc, 2))} | test loss {str(round(final_test_acc, 2))}')

Prices are :
 tensor([ 13.1860,  20.8905,  20.7936,   7.5810,   7.3815,  22.3250,   6.8400,
         14.0600,  18.7856,   5.2250,   5.4625,  15.9200,   5.4625,  15.9200,
         16.1289,   7.5050,  16.0693,  13.9200,  19.9500,  23.7405,  19.3800,
         13.1241,  18.7625,  17.8600,  18.7625,  17.8600,   4.1800,  21.5000,
         15.1000,  14.2405,  15.0670,  18.8860,   4.6360,   6.0800,   3.7050,
         16.5300,   3.7050,  16.5300,  14.4772,   5.0350,  34.2000,   9.5000,
         14.7250,  21.4700,  14.3250,  27.0750,   6.5550,   8.4550,   6.5550,
          8.4550,  14.3900,  20.6625,  27.3600,   7.0775,  15.1905,  17.5750,
          9.2625,  12.3569,  13.6315,  14.2405,  13.6315,  14.2405,  18.6500,
         16.6250,  13.9595,  11.5420,  13.2050,  39.2000,  19.9500,  16.3562,
         11.8206,  13.9270,  11.8206,  13.9270,  18.9050,  15.1141,  18.6200,
         18.9430,   5.5575,  20.9000,  20.3000,  28.5000,  11.8750,  16.9150,
         11.8750,  16.9150,  21.8310,   6.6500,   

From the above analyse, we can see that the rarity attributes do have relationship with their prices. And the data type is prety suitable for the MLP to deal with.

Buyers do tend to buy more fancy monkeys with interesting rare features, which really makes sense.